In [1]:
import pandas as pd
import numpy as np
# Visualization
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline
import datetime as dt

# The Project

There are Propert Tax Tranfer records for 1987 to 2010 which need to be transformed into a form that can be joined with current XML based schema in order to create the most comprehensive database as possible. These records date to the beginning of digitalized records and suffer from numerous problems due to their age (like the fact they face Y2K issues with their date format).  


# The Goal
At the end of this notebook there are two file produced, one with a list of property records, and the second is a list of buyers and sellers. These files are flat files, but due to their size they cannot be stored in Github so the notebook must be downloaded locally and run. Once run the two files will be saved as csv's.  

## Loading Records from 1987 to 2010
Once compiled into a single dataframe there are a total of 2,350,630 rows. Because of the format of the data there are multiple lines per property transfer, so the actual count of properties is only 681,377 which means on average there are 3 lines or more per property, indicating there is more than just a single seller and buyer.

The format of the original data is as a fixed-width text file which was converted in excel into columns based on the file specifications.  In the steps below those columns are adjusted and modified to match the current xml format as best as possible.  

## Issues With the Data

There are a lot of missing datapoints in the earlier records. Few had SPAN numbers and addresses of the actual properties were sparse in details, like for example, "OFF LAKE ST IN ADDISON VT" or "FORMER DOLPHIS SWEENEY & AVIS SWEENEY HOMEPLACE IN SO ALBANY VILLAGE". Given the lack of a SPAN number, or any GPS coordinates there is sustantial work to rectefy those records. For those who did have SPAN ids' they were in with other data in an acerage field and were split into their own column. 999-000-00000 was the span ID for 101,333 records, and 2,246,916 had no SPAN at all. Even in 2010 there were 4,400 rows that had 9999-00-00000 as the Span.

Monetary values were also stored as cents, in order to match the newer format all those values were converted to dollar amounts as a float.

In [27]:
# Load the csv version of the text files
df87 = pd.read_csv('PT1987.csv', header=None, low_memory=False)
df88 = pd.read_csv('PT1988.csv', header=None, low_memory=False)
df89 = pd.read_csv('PT1989.csv', header=None, low_memory=False)
df90 = pd.read_csv('PT1990.csv', header=None, low_memory=False)
df91 = pd.read_csv('PT1991.csv', header=None, low_memory=False)
df92 = pd.read_csv('PT1992.csv', header=None, low_memory=False)
df93 = pd.read_csv('PT1993.csv', header=None, low_memory=False)
df94 = pd.read_csv('PT1994.csv', header=None, low_memory=False)
df95 = pd.read_csv('PT1995.csv', header=None, low_memory=False)
df96 = pd.read_csv('PT1996.csv', header=None, low_memory=False)
df97 = pd.read_csv('PT1997.csv', header=None, low_memory=False)
df98 = pd.read_csv('PT1998.csv', header=None, low_memory=False)
df99 = pd.read_csv('PT1999.csv', header=None, low_memory=False)
df00 = pd.read_csv('PT2000.csv', header=None, low_memory=False)
df01 = pd.read_csv('PT2001.csv', header=None, low_memory=False)
df02 = pd.read_csv('PT2002.csv', header=None, low_memory=False)
df03 = pd.read_csv('PT2003.csv', header=None, low_memory=False)
df04 = pd.read_csv('PT2004.csv', header=None, low_memory=False)
df05 = pd.read_csv('PT2005.csv', header=None, low_memory=False)
df06 = pd.read_csv('PT2006.csv', header=None, low_memory=False)
df07 = pd.read_csv('PT2007.csv', header=None, low_memory=False)
df08 = pd.read_csv('PT2008.csv', header=None, low_memory=False)
df09 = pd.read_csv('PT2009.csv', header=None, low_memory=False)
df10 = pd.read_csv('PT2010.csv', header=None, low_memory=False)

# append all the years to the first year
df = df87.copy()
dflist = [df88, df89, df90, df91, df92, df93, df94, df95, df96, df97, df98, df99, df00, df01, df02, df03, df04, df05, df06, df07, df08, df09, df10]
df = df.append(dflist)

In [28]:
#Name and Add Correct Headers
col = ("Key Town Code", "Key Year/Month", "Key Year", "Key Document", "Key Record ID", "Update Count", "S/B", "Filler1", "Alt Key", "Name", "Street", 
       "City", "State", "Zip Code", "Country", "County/Town Code", "Return", "Number", "Development Rights", "Family Member Sale", "Financing", 
       "Purchased by Tenant", "Buyer Has Adjoining Prop", "Property Location 1", "Property Location Code", 
       "Transfer Type", "Acreage", "Frontage", "Depth", "Building Code 1", "Building Code 2", "Building Code 3", "Number of Apartments", 
       "Use Before", "Use After", "Exemption Code", "Total Selling Price", "Personal Property", "Real Value", "Tax Due", "Tax Paid", 
       "Special Circumstances", "Valid Sales Flag", "Acquired Date", "Filler2", "Category Code", "Per Cent Interest", "Filler3", "Land Gains Code 1", 
       "Land Gains Code 2", "Land Gains Code 3", "Span", "Act 291 Permit", "Act 291 Exempt", "Act 250 Permit", "Act 250 Exempt", "Town Name", "Leading0", 
       "Date of Record", "Book Number", "Page Number", "Listed Value", "Leading1","Listed Year", "Filler5", "Withholding Reason ", 
       "Certificate Number", "Batch Number", "leading2","Date Processed", "Incomplete Flag", "Sub Division", "Map/Parcel Number", "Date of Closing", "Filler6", 
       "Old Form Code", "Current Use Code", "Previously Occupied", "Residential Value", "Use Value", "Working Farm Value", "Filler7","unknown1")
df.columns = col

#Split out span and acreage from Filler4 area, named Span
df['Acreage-D2'] = df['Span'].str[13:22]
df['Span'] = df['Span'].str[:13]

#adjust acreage to correct decimel point
df['Acreage'] = df['Acreage']*.1
df['Acreage-D2'] = pd.to_numeric(df['Acreage-D2'])
df['Acreage-D2'] = df['Acreage-D2']*.01

# cuts out some empty columns
df = df[['Key Town Code', 'Key Year/Month', 'Key Year', 'Key Document', 'Key Record ID', 'Update Count', 'S/B', 'Alt Key', 'Name', 'Street', 'City', 'State', 'Zip Code', 'Country', 'County/Town Code',
         'Return', 'Number', 'Development Rights', 'Family Member Sale', 'Financing', 'Purchased by Tenant', 'Buyer Has Adjoining Prop', 'Property Location 1', 'Property Location Code', 'Transfer Type', 
         'Acreage', 'Frontage', 'Depth', 'Building Code 1', 'Building Code 2', 'Building Code 3', 'Number of Apartments', 'Use Before', 'Use After', 'Exemption Code', 'Total Selling Price', 
         'Personal Property', 'Real Value', 'Tax Due', 'Tax Paid', 'Special Circumstances', 'Valid Sales Flag', 'Acquired Date', 'Category Code', 'Per Cent Interest', 'Land Gains Code 1', 
         'Land Gains Code 2', 'Land Gains Code 3', 'Span', 'Act 291 Permit', 'Act 291 Exempt', 'Act 250 Permit', 'Act 250 Exempt', 'Town Name', 'Date of Record', 'Book Number', 'Page Number', 
         'Listed Value', 'Listed Year', 'Withholding Reason ', 'Certificate Number', 'Batch Number', 'Date Processed', 'Incomplete Flag', 'Sub Division', 'Map/Parcel Number', 'Date of Closing', 
         'Old Form Code', 'Current Use Code', 'Previously Occupied', 'Residential Value', 'Use Value','Working Farm Value',  'unknown1', 'Acreage-D2']]

# changes prices to be in dollars rather than cents
df['Total Selling Price'] = df['Total Selling Price']*0.01
df['Personal Property'] = df['Personal Property']*0.01
df['Real Value'] = df['Real Value']*0.01
df['Tax Due'] = df['Tax Due']*0.01
df['Tax Paid'] = df['Tax Paid']*0.01
df['Use Value'] = df['Use Value']*0.01

print("Total number of Records: ", len(df))
print("Total number of Properties: ", len(df[df['Key Record ID'] == 0]))

Total number of Records:  2350630
Total number of Properties:  681337


In [29]:
print("Group By Span, sorted by Count")
df['count'] = 1
df[['Span','count']].groupby(['Span'], as_index=0).sum().sort_values('count', ascending=0).head(10)

Group By Span, sorted by Count


,Span,count
86542,999-000-00000,101333
86541,998-000-00000,885
76776,690-219-13481,177
40045,363-112-13517,174
63973,588-185-10973,167
40078,363-112-13607,137
64496,588-185-13320,137
76762,690-219-13413,111
70628,621-195-13479,99
70739,621-195-13828,49


## Date

These records did not have a DNL field at all. In order to import into the modern form a DNL had to be created which required that all the dates be updated to 4 year digits. Additionaly given the age of these records they suffered some data integrety loss for dates, as dates were not done with a 4 digit format (YYYY) and some of those dates seem to be wildly inaccurate.  Additionally the importing of this data into excel then into Pandas meant the leading zeros were dropped so those had to be readded. 

There were four fields with data records, none have dates for all records. 
* Date Processed - 658265 records with dates
* Date of Record - 657050 records with dates 
* Date of Closing - 528926 records with dates 
* Acquired Date - 498122 records with dates

If a Date Processed had an odd value that does not match actual potential years, the 19000000 was put in place.

The Date of Closing also had some odd values that produced errors that had to be coerced. Some of the months were outside of 1-12 so for those it was defaulted to the month of 01

In [30]:
# convert the automatic float format to int in order to try to convert to the correct date
datefield = ['Date of Record','Date Processed','Date of Closing','Acquired Date']

for x in datefield:
    df[x] = df[x].fillna(0).astype(int)

In [31]:
df['DNL'] = df['Date Processed']

datefield = ['DNL','Date of Record','Date Processed','Date of Closing','Acquired Date']

# Due to leading zeros being dropped in import and that this has only a two digit date, determine and add the correct year 
def addyear(date):
    for x in date:
        if len(str(x)) == 3:
            y.append(int("20000" + str(x)))
        elif len(str(x)) == 4:
            y.append(int("2000" + str(x)))
        elif len(str(x)) == 5:
             y.append(int("200" + str(x)))
        elif (len(str(x)) == 6) & (int(x) >= 100101) & (int(x) <= 101231):
            y.append(int("20" + str(x)))
        elif (len(str(x)) == 6) & (int(x) > 101231) & (int(x) <= 991231):
            y.append(int("19" + str(x)))
        else:
            y.append(0)

for x in datefield:
    y=[]
    addyear(df[x])
    df[x] = y

In [32]:
# Change the dates into "-" split dates to it could be converted to datetime format and match xml
def splitdate(date):
    for x in date:
        if x == 0:
            y.append(x)
        else:
            x = str(x)
            y.append(x[0:4] + "-" + x[4:6] + "-" +  x[6:8])

y = []
splitdate(df['Date Processed'])
df['Date Processed'] = y

y = []
splitdate(df['Date of Record'])
df['Date of Record'] = y

y = []
splitdate(df['Date of Closing'])
df['Date of Closing'] = y

y = []
splitdate(df['Acquired Date'])
df['Acquired Date'] = y

## DNL

None of these records had DNL ids, a DNL record was create from the Date Processed, Key Document number and for trailing zeros. These seems to match the format of later records as best as possible and should provide a unique not duplicated by later records. There is also a record under Key Document that does not have a value but rather the fragement of a word "S MADE" which is replaces by "100000", if there is no document ID it is also set to "100000"

In [33]:
y = []
for x in df['Key Document']:
    if x == 'S MADE':
         y.append(100000)
    elif (len(str(x)) ==  6):
        y.append(x)
    else:
        y.append(100000)
df['Key Document'] = y        
df['Key Document'] = df['Key Document'].astype(int)

In [34]:
# create a dnl number with Date Processed and Key Document number        
df['DNL']= df['DNL'].astype(float)
df['DNL'] = df['DNL'].replace(0.0,np.nan)
df['DNL'].fillna(method='ffill', inplace=True)
df['DNL']= df['DNL'].astype(int)
df['DNL'] = df['DNL'].astype(str) 

x = []
for num in df['DNL']:
    x.append(num + '0000')
df['DNL'] = x    
df['Key Document'] = df['Key Document'].astype(str)    
df['DNL'] = df['DNL'] +  df['Key Document']


# Converting Financing section into Columns
Certain types of records were stored as numeric values rather than binary columns. In oder for it tom 
* Space=Unknown
* 1=Bank
* 2=Owner
* 3=Other

In [36]:
# fixing bad data entry and replacing 9 with zero as it is not clear what that would stnad for
df['Financing'].replace("O",0, inplace=True)
df['Financing'].replace(9,0, inplace=True)
df['Financing'] = df['Financing'].fillna(0).astype(int)

financing = pd.get_dummies(df['Financing'])
financing.columns = ['Unknown','Bank','Owner','Other']
df = pd.concat([df,financing], axis=1, sort=False)

# Property File

In [37]:
proponlydf = df[df['Key Record ID'] == 0]
len(proponlydf)

681337

In [38]:
# convert to datetime felts
#proponlydf['Date Processed'] = pd.to_datetime(proponlydf['Date Processed'], infer_datetime_format=True)
#proponlydf['Date of Record'] = pd.to_datetime(proponlydf['Date of Record'], infer_datetime_format=True)
#proponlydf['Acquired Date'] = pd.to_datetime(proponlydf['Acquired Date'], infer_datetime_format=True)
#proponlydf['Date of Closing'] = pd.to_datetime(proponlydf['Date of Closing'], infer_datetime_format=True)

In [40]:
columnname = ['receivedDate','postedDate','DLN','totalSellers','totalBuyers','internationalAddress','propertyLocationStreet','propertyLocationCity',
              'closingDate’,’interestPropertyType','interestUndivPercent','landSize','specialFactorsDevRights','specialFactorsRelationship',
              'financingBank','financingOwner','financingOther','buildingTypeNone','buildingTypeSingle','buildingTypeSeasonal','buildingTypeFarm',
              'buildingTypeMultiFamily','buildingTypeMultiUnit','buildingTypeMobile','buildingTypeCondo','buildingTypeConUnit','buildingTypeFactory',
              'buildingTypeStore','buildingTypeNewCons','buildingTypeOther','sellerUseOfProperty','rentedBefore','buyerUseOfProperty','specialCircumCode',
              'legacyBuyerUseOfProperty','legacySellerUseOfProperty','rentedAfter','tenantPurchase','buyerAdjoiningProperty','currentUseValueLien',
              'currentUseEnrollmentContinue','propertyTaxExemption','totalPricePaid','personalPropertyPricePaid','realPropertyPricePaid',
              'principalResidenceValue','currentUseMarketValue','qualifiedFarmValue','taxDue','dateSellerAcquired','noLandGainsTaxReturn1',
              'noLandGainsTaxReturn2','noLandGainsTaxReturn3','city','dateOfRecord','listedValue','parcelIDNo','grandListCategory','span',
              'townCode','schoolCode','countyCode']
propdf = pd.DataFrame(columns=columnname)

## Missing Data
Any fields commented out are missing that data at this point. For the building type some could be created from other data fields, the key is at the end of this document

In [41]:
propdf['receivedDate'] = proponlydf['Date of Record']
propdf['postedDate'] = proponlydf['Date Processed']
propdf['DLN'] = proponlydf['DNL']
#propdf['totalSellers'] = proponlydf['']
#propdf['totalBuyers'] = proponlydf['']
propdf['propertyLocationStreet'] = proponlydf['Property Location 1']
propdf['propertyLocationCity'] = proponlydf['Town Name']
propdf['closingDate'] = proponlydf['Date of Closing']
#propdf['interestPropertyType'] = proponlydf['']
propdf['interestUndivPercent'] = proponlydf['Per Cent Interest']
propdf['landSize'] = proponlydf['Acreage']
propdf['specialFactorsDevRights'] = proponlydf['Development Rights']
propdf['specialFactorsRelationship'] = np.nan
propdf['financingBank'] = proponlydf['Bank']
propdf['financingOwner'] = proponlydf['Owner']
propdf['financingOther'] = proponlydf['Other']
#propdf['buildingTypeNone'] = proponlydf['']
#propdf['buildingTypeSingle'] = proponlydf['']
#propdf['buildingTypeSeasonal'] = proponlydf['']
#propdf['buildingTypeFarm'] = proponlydf['']
#propdf['buildingTypeMultiFamily'] = proponlydf['']
#propdf['buildingTypeMultiUnit'] = proponlydf['']
#propdf['buildingTypeMobile'] = proponlydf['']
#propdf['buildingTypeCondo'] = proponlydf['']
#propdf['buildingTypeConUnit'] = proponlydf['']
#propdf['buildingTypeFactory'] = proponlydf['']
#propdf['buildingTypeStore'] = proponlydf['']
#propdf['buildingTypeNewCons'] = proponlydf['']
#propdf['buildingTypeOther'] = proponlydf['']
#propdf['sellerUseOfProperty'] = proponlydf['']
propdf['rentedBefore'] = np.nan
propdf['buyerUseOfProperty'] = proponlydf['Use Before']
propdf['specialCircumCode'] = proponlydf['Special Circumstances']
propdf['legacyBuyerUseOfProperty'] = proponlydf['Use After']
propdf['legacySellerUseOfProperty'] = proponlydf['Use Before']
propdf['rentedAfter'] =  np.nan
propdf['tenantPurchase'] = proponlydf['Purchased by Tenant']
propdf['buyerAdjoiningProperty'] = proponlydf['Buyer Has Adjoining Prop']
propdf['currentUseValueLien'] = np.nan
propdf['currentUseEnrollmentContinue'] = np.nan
#propdf['propertyTaxExemption'] = proponlydf['Withholding Reason']
propdf['totalPricePaid'] = proponlydf['Total Selling Price']
propdf['personalPropertyPricePaid'] = proponlydf['Personal Property']
propdf['realPropertyPricePaid'] = proponlydf['Real Value']
propdf['principalResidenceValue'] = np.nan
propdf['currentUseMarketValue'] = proponlydf['Use Value']
propdf['qualifiedFarmValue'] = proponlydf['Working Farm Value']
propdf['taxDue'] = proponlydf['Tax Due']
propdf['dateSellerAcquired'] = proponlydf['Acquired Date']
propdf['noLandGainsTaxReturn1'] = proponlydf['Land Gains Code 1']
propdf['noLandGainsTaxReturn2'] = proponlydf['Land Gains Code 2']
propdf['noLandGainsTaxReturn3'] = proponlydf['Land Gains Code 3']
propdf['city'] = proponlydf['City']
propdf['dateOfRecord'] = proponlydf['Date of Record']
propdf['listedValue'] = proponlydf['Listed Value']
propdf['parcelIDNo'] = proponlydf['Map/Parcel Number']
propdf['grandListCategory'] = np.nan
propdf['span'] = proponlydf['Span']
#propdf['townCode'] = proponlydf['']
#propdf['schoolCode'] = proponlydf['']
#propdf['countyCode'] = proponlydf['']


In [42]:
propdf.head()

,receivedDate,postedDate,DLN,totalSellers,totalBuyers,internationalAddress,propertyLocationStreet,propertyLocationCity,"closingDate’,’interestPropertyType",interestUndivPercent,...,city,dateOfRecord,listedValue,parcelIDNo,grandListCategory,span,townCode,schoolCode,countyCode,closingDate
0,1987-01-26,1987-08-21,198708210000500026,NaN,NaN,NaN,RTE 17 ADDISON,ADDISON,NaN,0.0,...,VERGENNES,1987-01-26,2.24e+06,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1987-02-13,1987-08-21,198708210000500018,NaN,NaN,NaN,RTE 17 ADDISON,ADDISON,NaN,0.0,...,PT PLEASANT,1987-02-13,7.58e+06,NaN,NaN,NaN,NaN,NaN,NaN,0
8,1987-02-24,1988-06-14,198806140000500024,NaN,NaN,NaN,OFF LAKE ST IN ADDISON VT,ADDISON,NaN,0.0,...,BRISTOL,1987-02-24,1.14e+06,NaN,NaN,NaN,NaN,NaN,NaN,0
12,1987-03-18,1987-08-21,198708210000500016,NaN,NaN,NaN,TOWN HIGHWAY 1 AND VT RT 17 IN ADDISON VT,ADDISON,NaN,0.0,...,VERGENNES,1987-03-18,0,NaN,NaN,NaN,NaN,NaN,NaN,0
15,1987-03-15,1987-08-21,198708210000500017,NaN,NaN,NaN,ROUTE 22A ADDISON,ADDISON,NaN,0.0,...,VERGENNES,1987-03-15,2.46e+06,NaN,NaN,NaN,NaN,NaN,NaN,0


In [43]:
propdf.to_csv("property.csv")

CATEGORY CODES
* 00 UNKNOWN
* 01 Residential less than 6 acres
* 02 Residential 6 acres or more
* 03 Mobile Home without land
* 04 Mobile Home with land
* 05 Vacation less than 6 acres
* 06 Vacation 6 acres or more
* 07 Commercial
* 08 Not Used
* 09 Industrial
* 10 Utilities Electric
* 11 Utilities Other
* 12 Farms
* 13 Government Land
* 14 Timber/Wood Land
* 15 Miscellaneous/Open Land


SPECIAL CIRCUMSTANCES CODES
* 00 None
* 01 Price Paid less than value of land
* 02 Straw Transfer
* 03 In Lieu of Foreclosure
* 04 Real Property is estimate by Tax Dept.
* 05 One of Several Returns for one Property
* 06 Contract Sales
* 07 Burned-out Buildings
* 08 Sales between members of the immediate family
* 09 Sales between a corporation and a stockholder
* 10 Tax sales; Sheriff's sales; Bankruptcy, Receivership, Dissolution or Liquidation sales 
* 11 Sales by Guardians, Trustees, Executors, and Administrators
* 12 Sales to or from the U.S. Government, the State of  Vermont or any Political subdivision of Vermont
* 13 Sales to or from any Charitable,  Religious or Benevolent organization
* 14 Sales where unusual financing significantly affected sale price
* 15 Sales where all assessed interests were not sold, thereby affecting sale price
* 16 Sales of Property assessed in more than one town
* 17 Any sales that include personal property unless the amount of the personal can be determined and reported
* 18 Sales of Property conveying only a portion of the assessed unit, such as a lot or lots sold off from a farm
* 19 Sales where the property sold was substantially changed assessment date but prior to date of sale
* 20 Other reasons

VALID SALES FLAG

The Valid Sales Flag is determined by the type of Special Circumstances Code used.

CODE Special Circumstances Code
* 01 00,01
* 02 05,07,11,12,16,18,19,20
* 03 02,03,04,06,08,09,10,13,14,15,17



In [44]:
bsdf = df[df['Key Record ID'] != 0]
len(bsdf)

1669293

In [46]:
bsdf.head()

,Key Town Code,Key Year/Month,Key Year,Key Document,Key Record ID,Update Count,S/B,Alt Key,Name,Street,...,Use Value,Working Farm Value,unknown1,Acreage-D2,count,DNL,Unknown,Bank,Owner,Other
1,3,8701,88,500026,10,0,S,8850002610,SLEEPER LINDA,RD 3 BOX 200,...,NaN,NaN,NaN,NaN,1,198708210000500026,1,0,0,0
2,3,8701,88,500026,11,0,B,8850002611,SLEEPER JEFFREY,RD 3 BOX 200,...,NaN,NaN,NaN,NaN,1,198708210000500026,1,0,0,0
3,3,8701,88,500026,12,0,B,8850002612,SLEEPER LINDA,RD 3 BOX 200,...,NaN,NaN,NaN,NaN,1,198708210000500026,1,0,0,0
5,3,8702,88,500018,10,0,S,8850001810,GALLAS NESTA M,BOX 450,...,NaN,NaN,NaN,NaN,1,198708210000500018,1,0,0,0
6,3,8702,88,500018,11,0,B,8850001811,COSGROVE JR KEVIN M,RD 3 BOX 248,...,NaN,NaN,NaN,NaN,1,198708210000500018,1,0,0,0


In [47]:
bsdf = bsdf[['DNL','Key Record ID', 'Update Count', 'S/B', 'Alt Key', 'Name', 'Street',
       'City', 'State', 'Zip Code', 'Country']]
bsdf.head()

,DNL,Key Record ID,Update Count,S/B,Alt Key,Name,Street,City,State,Zip Code,Country
1,198708210000500026,10,0,S,8850002610,SLEEPER LINDA,RD 3 BOX 200,VERGENNES,VT,5491,NaN
2,198708210000500026,11,0,B,8850002611,SLEEPER JEFFREY,RD 3 BOX 200,VERGENNES,VT,5491,NaN
3,198708210000500026,12,0,B,8850002612,SLEEPER LINDA,RD 3 BOX 200,VERGENNES,VT,5491,NaN
5,198708210000500018,10,0,S,8850001810,GALLAS NESTA M,BOX 450,PT PLEASANT,PA,18950,NaN
6,198708210000500018,11,0,B,8850001811,COSGROVE JR KEVIN M,RD 3 BOX 248,VERGENNES,VT,5491,NaN


In [48]:
bsdf.to_csv("buyerseller.csv")